## Creating Parameters

In [0]:
init_load = int(dbutils.widgets.get("init_load"))

In [0]:
dbutils.widgets.text("init_load", "")
init_load = int(dbutils.widgets.get("init_load"))
from pyspark.sql.functions import *

## Data Reading

In [0]:
df = spark.sql(
    '''
    SELECT * FROM dtbproject.silver.customers
    '''
)

In [0]:
df.display()

## Dropping Duplicates Based On Customer ID

In [0]:
df = df.dropDuplicates(subset=['customer_id'])

## Dividing Old Vs New Records

Creating Pseudo Table

In [0]:
if init_load == 0:
  df_olds = spark.sql(
    '''
    SELECT DimCustomerKey, customer_id, create_date, update_date FROM dtbproject.gold.DimCustomer
    '''
  )
else:
  df_olds = spark.sql(
    '''
    SELECT 0 Old_DimCustomerkey, 0 customer_id_old, 0 create_date_old, 0 update_date_old FROM dtbproject.silver.customers WHERE 1=0'''
  )


In [0]:
df_olds.display()

## Applying Joins to our Old Records

In [0]:
df_join = df.join(df_olds, df.customer_id == df_olds.customer_id_old, "left")
df_join.display()

In [0]:
df_news = df_join.filter(col("old_DimCustomerKey").isNull())

df_olds = df_join.filter(col("old_DimCustomerKey").isNotNull())


Preparing DF OLD

In [0]:

#Dropping Less Required Columns

df_olds = df_olds.drop("Old_DimCustomerkey", "customer_id_old","update_date_old" )

#Renaming "create_date_old" to "create_date" a
df_olds = df_olds.withColumnRenamed("create_date_old", "create_date")
df_olds = df_olds.withColumn("create_date", to_timestamp(col("create_date")))
#Adding "update_date" column with current date
df_olds = df_olds.withColumn("update_date", current_timestamp())
df_olds.display()

In [0]:
df_olds.display()

Preparing Df_New

In [0]:
df_news.display()

In [0]:
df_news = df_news.drop("Old_DimCustomerKey", "customer_id_old", "create_date_old", "update_date_old")
#Recreating "create_date" and "update_date" column with timestamp()
df_news = df_news.withColumn("create_date", current_timestamp())
df_news = df_news.withColumn("update_date", current_timestamp())
df_news.display()

##Surrogate Key From 1

In [0]:
df_news = df_news.withColumn("DimCustomerKey", monotonically_increasing_id()+lit(1))
df.display()

In [0]:
df_news.display()

# Adding Max Surrogate Key

In [0]:
if init_load == 1:
  max_surrogate_key = 0
else:
    df_maxsur = spark.sql("""
                          SELECT MAX(DimCustomerKey) AS max_surrogate_key FROM dtbproject.gold.DimCustomer
                          """)
    #Converting df_maxsur to max_surrogate_key
    max_surrogate_key = df_maxsur.collect()[0]['max_surrogate_key']
    

In [0]:
df_news = df_news.withColumn("DimCustomerKey", lit(max_surrogate_key) + col("DimCustomerKey"))


In [0]:
df_news.display()

# Union Of Df_Old And Df_New

In [0]:

df_final = df_olds.unionByName(df_news)


SCD Type 1 -> UPSERT

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("dtbproject.gold.DimCustomer"):
    dlt_obj = DeltaTable.forPath(spark, "abfss://gold@azuredatabricksjesslake.dfs.core.windows.net/DimCustomer")
    dlt_obj.alias("trg").merge(df_final.alias("src"),"trg.DimCustomerKey = src.DimCustomerKey")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()
   
else:
     df_final.write.format("delta").mode("overwrite").option("path", "abfss://gold@azuredatabricksjesslake.dfs.core.windows.net/DimCustomer").saveAsTable("dtbproject.gold.DimCustomer")
    

In [0]:
%sql
